## Сбор данных 

In [1]:
import requests
import numpy as np
import pandas as pd
import yaml

**Считываем токен из `.yaml` файла:**

In [2]:
conf = open('config.yaml')

config = yaml.load(conf, yaml.SafeLoader)

access_token = config['access_token']

**Считываем описание признаков из `.yaml` файла:**

In [125]:
descr = open('description_of_features.yaml', mode = 'r', encoding = 'utf-8')

description = yaml.load(descr, yaml.SafeLoader)

**Подготовка к API запросу. Используется метод `users.get`:**

In [136]:
url = 'https://api.vk.com/method/users.get?'

params = {
    'v': 5.131,
    'access_token': access_token,
    'fields': 'sex, bdate, books, counters, games, interests, music, movies, personal, universities, relation'
}

user_id_list = np.random.randint(1, 60000000, 10)

**Функция, которая 'раскрывает' словарь:**

In [5]:
def del_dict(dictionary, del_dict):
        
    if not del_dict in dictionary.keys():
        return

    if type(dictionary[del_dict]) != list:
        for key in dictionary[del_dict].keys():
            dictionary[del_dict + '_' + key] = dictionary[del_dict][key]
        
    elif len(dictionary[del_dict]) != 0:
        for key in dictionary[del_dict][0].keys():
            dictionary[del_dict + '_' + key] = dictionary[del_dict][0][key]
            
    dictionary.pop(del_dict)    
    
    return dictionary

**Функция, которя возрващает `DateFrame` путем API запросов:**

In [137]:
def get_data(count=1000):
    
    user_str = ''

    user_id_list = np.random.randint(1, 60000000, count)

    for user_id in user_id_list:
        user_str = user_str + str(user_id) + ', '
    
    df = pd.DataFrame(columns=description.keys())
    
    params['user_ids'] = user_str
        
    r = requests.get(url, params = params)
    
    data_dict = r.json()['response']
    
    for user_dict in data_dict:
        
        if  (user_dict['first_name'] != 'DELETED') and \
            ('deactivated' not in user_dict.keys()) and \
            (user_dict['is_closed'] == False) and \
            (user_dict['sex'] != 0):
        
            del_dict(user_dict, 'counters')
            del_dict(user_dict, 'personal')
            del_dict(user_dict, 'universities')
            del_part(user_dict, 'relation_partner')
            
            df = df.append(user_dict, ignore_index=True)
            
    return df

In [138]:
def del_part(dictionary, del_dict):
    
    if not del_dict in dictionary.keys():
        return
    
    dictionary.pop(del_dict)
    
    return dictionary

In [139]:
%%time

full_df = pd.DataFrame(columns=description.keys())

while len(full_df) < 100:
    
    df = get_data()
    
    full_df = pd.concat([full_df, df])

Wall time: 5.62 s


In [164]:
def get_users_csv(min_count, file_name):
    
    file_name = file_name + '.csv' 
    
    full_df = pd.DataFrame(columns=description.keys())
    
    while len(full_df) < min_count:
    
        df = get_data()
    
        full_df = pd.concat([full_df, df])
        
        full_df.to_csv(file_name)
        
    return full_df

In [173]:
%%time

df = get_users_csv(25000, 'data/data_2')

PermissionError: [Errno 13] Permission denied: 'data/data_2.csv'